## Import libaries

In [1]:
# if installed the model, skip the cell.
# import sys

# sys.path.append('/home/jovyan/wa/u23/crop-mask/dea_ml')

In [2]:
import json
import joblib
from dea_ml.core.feature_layer import merge_tile_ds, get_xy_from_task

from dea_ml.helpers.json_to_taskstr import extract_taskstr_from_geojson
from dea_ml.helpers.io import download_file
from dea_ml.core.africa_geobox import AfricaGeobox
from dea_ml.core.predict_from_feature import PredictFromFeature
from dea_ml.core.post_processing import post_processing
from dea_ml.config.product_feature_config import FeaturePathConfig

## Configuration was done with the following class

In [3]:
# import the configuration

config = FeaturePathConfig

config

dea_ml.config.product_feature_config.FeaturePathConfig

## Analysis Params


In [4]:
tile_geojson_url= config.tiles_geojson #"s2_tiles_eastern_aez.geojson"
print(tile_geojson_url)
#from the notebooks
ml_model_url = config.model_path #'gm_mads.joblib'
print(ml_model_url)

#edit this outside the notebook and import
# config_stuff = 'config.yml'

https://github.com/digitalearthafrica/crop-mask/blob/main/eastern_cropmask/data/s2_tiles_eastern_aez.geojson?raw=true
https://github.com/digitalearthafrica/crop-mask/blob/main/eastern_cropmask/results/gm_mads_two_seasons_ml_model_20210301.joblib?raw=true


In [5]:
local_file = download_file(tile_geojson_url, '/tmp/tiles_geojson.json')
with open(local_file) as fh:
    tiles_geojson_dict = json.load(fh)

In [6]:
tiles_geojson_dict['features'][0].keys()

dict_keys(['type', 'properties', 'geometry'])

## Generate 'tasks'


In [7]:
tasks = extract_taskstr_from_geojson(time_range='2019-01--P6M', geojson=tiles_geojson_dict)

In [8]:
tasks[:5]

['x+029/y+000/2019-01--P6M',
 'x+029/y-001/2019-01--P6M',
 'x+029/y-002/2019-01--P6M',
 'x+029/y-003/2019-01--P6M',
 'x+029/y-004/2019-01--P6M']

# load prepared model

In [9]:
local_model_path = download_file(ml_model_url, '/tmp/ml-model-sample')

In [10]:
model = joblib.load(local_model_path)
model

RandomForestClassifier(criterion='entropy', max_features='log2',
                       n_estimators=400, n_jobs=31, random_state=1)

## Define feature layer and post process functions

Or import a python file is preferred

In [11]:
# get the task
tasks[0]

'x+029/y+000/2019-01--P6M'

In [12]:
x, y = get_xy_from_task(tasks[0])
x, y

(29, 0)

In [13]:
geobox_dict = AfricaGeobox().geobox_dict

## gm_ds is the geomedian prepared with our indexing methods
- If not available, just set it as none. then it will be collected from the tifs manually with odc-stats command

In [14]:
subfld, geobox, data = merge_tile_ds(x, y, config, geobox_dict, gm_ds=None) 

/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)
/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)
/env/lib/python3.6/site-packages/xarray/core/computation.py:700: RuntimeWarning: divide by zero encountered 

In [15]:
data

<xarray.Dataset>
Dimensions:        (x: 4800, y: 4800)
Coordinates:
  * y              (y) float64 9.599e+04 9.597e+04 9.595e+04 ... 50.0 30.0 10.0
  * x              (x) float64 2.784e+06 2.784e+06 ... 2.88e+06 2.88e+06
    spatial_ref    int32 6933
Data variables:
    nir_S1         (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_S1         (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    blue_S1        (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    swir_1_S1      (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_edge_2_S1  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    nir_narrow_S1  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_edge_1_S1  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    edev_S1        (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    bcdev_S1       (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    green_S1       (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    sdev_S1        (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    swir_2_S1      (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_edge_3_S1  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    NDVI_S1        (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    LAI_S1         (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    MNDWI_S1       (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    rain_S1        (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    swir_1_S2      (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    blue_S2        (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_edge_2_S2  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_S2         (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    nir_S2         (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    swir_2_S2      (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    sdev_S2        (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    green_S2       (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_edge_3_S2  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    bcdev_S2       (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    red_edge_1_S2  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    edev_S2        (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    nir_narrow_S2  (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    NDVI_S2        (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    LAI_S2         (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    MNDWI_S2       (y, x) float64 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    rain_S2        (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>
    slope          (y, x) float32 dask.array<chunksize=(4800, 4800), meta=np.ndarray>

In [16]:
subfld, geobox

('x+029/y+000',
 GeoBox(4800, 4800, Affine(20.0, 0.0, 2784000.0,
        0.0, -20.0, 96000.0), epsg:6933))

# Run prediction

In [17]:
pff = PredictFromFeature(config, geobox_dict)

In [18]:
predicted = pff.predict_with_model(config, model, data)

   predicting...
   probabilities...


In [19]:
predicted.persist()

<xarray.Dataset>
Dimensions:        (x: 4800, y: 4800)
Coordinates:
  * x              (x) float64 2.784e+06 2.784e+06 ... 2.88e+06 2.88e+06
  * y              (y) float64 9.599e+04 9.597e+04 9.595e+04 ... 50.0 30.0 10.0
    spatial_ref    int32 0
Data variables:
    Predictions    (y, x) int64 dask.array<chunksize=(100, 4800), meta=np.ndarray>
    Probabilities  (y, x) float64 dask.array<chunksize=(100, 4800), meta=np.ndarray>
Attributes:
    grid_mapping:  spatial_ref

# Post processing

In [20]:
# there some minor issue with dask_ml functions to raise the warning.
predict = post_processing(data, predicted, config, geobox)
prob = predicted.Probabilities

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.95.117.28/4044/1 remote=inproc://10.95.117.28/4044/9>


# save data
- result will be in: 
```/home/jovyan/wa/u23/data/crop_mask_eastern/v0.1.7/x+029/y+000/2019```

In [21]:
pff.save_data(subfld, predict, prob, geobox)

[2021-03-23 06:29:21,271] {predict_from_feature.py:118} INFO - collecting mask and write cog.
[2021-03-23 06:30:02,491] {env.py:433} WARNING - CPLE_NotSupported in driver GTiff does not support creation option WIDTH
[2021-03-23 06:30:02,492] {env.py:433} WARNING - CPLE_NotSupported in driver GTiff does not support creation option HEIGHT
[2021-03-23 06:30:02,493] {env.py:433} WARNING - CPLE_NotSupported in driver GTiff does not support creation option COUNT
[2021-03-23 06:30:02,493] {env.py:433} WARNING - CPLE_NotSupported in driver GTiff does not support creation option DTYPE
[2021-03-23 06:30:02,493] {env.py:433} WARNING - CPLE_NotSupported in driver GTiff does not support creation option CRS
[2021-03-23 06:30:02,494] {env.py:433} WARNING - CPLE_NotSupported in driver GTiff does not support creation option TRANSFORM
[2021-03-23 06:30:04,521] {predict_from_feature.py:125} INFO - collecting prob and write cog.
[2021-03-23 06:30:04,647] {env.py:433} WARNING - CPLE_NotSupported in driver 

In [22]:
!ls /home/jovyan/wa/u23/data/crop_mask_eastern/v0.1.7/x+029/y+000/2019

crop_mask_eastern_x+029_y+000_2019.json
crop_mask_eastern_x+029_y+000_2019_mask.tif
crop_mask_eastern_x+029_y+000_2019_prob.tif


## Either in this notebook or a seperate one, do the next steps: 

1. Database building
2. AWS syncing to S3

For command line operations in a notebook we can run commands using magic:
        
        e.g. !aws s3 mb s3://deafrica-data-dev-af --endpoint-url=htp://192.168.0.19:4566